In [40]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

In [41]:
tag = 'christmasbreak'
browser = webdriver.Chrome('/Users/matt/Downloads/chromedriver')
browser.get('https://twitter.com/hashtag/' + tag + '?lang=en')

In [42]:
links=[]

def scrape(start, end):
    print('scrolling: {}-{}'.format(start, end))
    Pagelength = browser.execute_script("window.scrollTo({}, {});".format(start, end))
    source = browser.page_source
    data=bs(source, 'html.parser')
    body = data.find('body')
    script = body.findAll('div', {'data-permalink-path': True})
    for div in script:
        status = div.get('data-permalink-path')
        links.append('https://www.twitter.com{}'.format(status))
    
    time.sleep(5)
    if (end < 500000):
        scrape(start + 3000, end + 3000)

In [43]:
scrape(0, 3000)

scrolling: 0-3000
scrolling: 3000-6000
scrolling: 6000-9000
scrolling: 9000-12000
scrolling: 12000-15000
scrolling: 15000-18000
scrolling: 18000-21000
scrolling: 21000-24000
scrolling: 24000-27000
scrolling: 27000-30000
scrolling: 30000-33000
scrolling: 33000-36000
scrolling: 36000-39000
scrolling: 39000-42000
scrolling: 42000-45000
scrolling: 45000-48000
scrolling: 48000-51000
scrolling: 51000-54000
scrolling: 54000-57000
scrolling: 57000-60000
scrolling: 60000-63000
scrolling: 63000-66000
scrolling: 66000-69000
scrolling: 69000-72000
scrolling: 72000-75000
scrolling: 75000-78000
scrolling: 78000-81000
scrolling: 81000-84000
scrolling: 84000-87000
scrolling: 87000-90000
scrolling: 90000-93000
scrolling: 93000-96000
scrolling: 96000-99000
scrolling: 99000-102000
scrolling: 102000-105000
scrolling: 105000-108000
scrolling: 108000-111000
scrolling: 111000-114000
scrolling: 114000-117000
scrolling: 117000-120000
scrolling: 120000-123000
scrolling: 123000-126000
scrolling: 126000-129000
sc

In [44]:
unique = list(set(links))
print('Found {} tweets'.format(len(unique)))

Found 1797 tweets


In [45]:
result = pd.DataFrame()
cols = ['user', 'location', 'geoLocation', 'tweet', 'timeStamp', 'replies', 'retweets', 'favorites', 'url']

for i in range(len(unique)):
    try:
        print(i, end=' ')
        page = urlopen(unique[i]).read()
        data=bs(page, 'html.parser')
        body = data.find('body')
        user = body.find('b', {'class': 'u-linkComplex-target'})
        location = body.find('span', {'class': 'ProfileHeaderCard-locationText u-dir'})
        content = body.find('p', {'class': 'tweet-text'})
        tweetDetails = body.find('div', {'class': 'tweet-details-fixer'})
        timeStamp = tweetDetails.find('span').find('span').text
        
        geoLocation = body.find('span', {'class': 'permalink-tweet-geo-text'})
        gl = '' if geoLocation == None else geoLocation.find('a').text
        
        replyContainer = body.find('div', {'class': 'ProfileTweet-action--reply'})
        replies = replyContainer.find('span', {'class': 'ProfileTweet-actionCount'}).find('span').text
        
        retweetContainer = body.find('div', {'class': 'ProfileTweet-action--retweet'})
        retweets = retweetContainer.find('span', {'class': 'ProfileTweet-actionCount'}).find('span').text
        
        favoriteContainer = body.find('div', {'class': 'ProfileTweet-action--favorite'})
        favorites = favoriteContainer.find('span', {'class': 'ProfileTweet-actionCount'}).find('span').text
        
        tweet = {
            'user': '@{}'.format(user.text),
            'location': location.text.strip(),
            'geoLocation': gl,
            'tweet': content.text,
            'timeStamp': timeStamp,
            'replies': replies or 0,
            'retweets': retweets or 0,
            'favorites': favorites or 0,
            'url': unique[i]
        }
        x = pd.DataFrame.from_dict(json_normalize(tweet), orient='columns') 
        x = x[cols]
        result = result.append(x) 
    except:
        np.nan

result.index = range(len(result.index))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [46]:
result = result.drop_duplicates()
print('scraped {} tweets'.format(len(result)))
result.head()

scraped 1010 tweets


,user,location,geoLocation,tweet,timeStamp,replies,retweets,favorites,url
0,@Moore_Of_Hannah,,,Time to start on my almost completely blank Cr...,8:57 AM - 4 Jan 2015,0,3,7,https://www.twitter.com/Moore_Of_Hannah/status...
1,@AlexisMarie268,"Plainview, Texas",,Good times with @crawford_dakota @BrennenLeFev...,10:27 PM - 19 Dec 2014,1,3,5,https://www.twitter.com/AlexisMarie268/status/...
2,@Jack_McLeay,,,Xmas break has been #crazy ! I've hugged like ...,10:19 PM - 22 Dec 2013,1,1,13,https://www.twitter.com/Jack_McLeay/status/415...
3,@ChooseHSUTX,"Abilene, Texas",,Current Cowboys: Finals are over! Drive safe a...,10:31 AM - 8 Dec 2016,0,8,19,https://www.twitter.com/ChooseHSUTX/status/806...
4,@haleybergan,,,This week has full permission to fly by. #chri...,7:14 PM - 16 Dec 2012,0,3,5,https://www.twitter.com/haleybergan/status/280...


In [47]:
result.to_csv('output.csv', columns=cols, index=None)